In [1]:
import numpy
import tensorflow
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
train_path = 'C:/Users/noufa/Downloads/Projs/Pneumonia_Chest_Xray/chest_xray/train/'
test_path = 'C:/Users/noufa/Downloads/Projs/Pneumonia_Chest_Xray/chest_xray/test/'
val_path = 'C:/Users/noufa/Downloads/Projs/Pneumonia_Chest_Xray/chest_xray/val/'

### CNN Model

In [2]:

cnn = Sequential()

#Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))

#Pooling
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu"))

# 2nd Pooling layer
cnn.add(MaxPooling2D(pool_size = (2, 2)))

#3rd Convolution
cnn.add(Conv2D(32,(3,3),activation='relu'))

#3rd Pooling Layer
cnn.add(MaxPooling2D(pool_size=(2,2)))

# Flatten the layer
cnn.add(Flatten())

# Fully Connected Layers
cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'sigmoid', units = 1))

# Compile the Neural network
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [3]:
num_of_test_samples = 600
batch_size = 32

In [11]:
# Fitting the CNN to the images
# The function ImageDataGenerator augments your image by iterating through image as your CNN is getting ready to process that image

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)  #Image normalization.

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(val_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

In [6]:
cnn_model = cnn.fit_generator(training_set,
                         steps_per_epoch = 163,
                         epochs = 1,
                         validation_data = validation_generator,
                         validation_steps = 624)

C:\Users\noufa\Anaconda3\envs\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


163/163 [==============================] - 273s 2s/step - loss: 0.5009 - accuracy: 0.7763 - val_loss: 0.5378 - val_accuracy: 0.6875


In [7]:
test_accu = cnn.evaluate_generator(test_set,steps=624)
print('The testing accuracy is :',test_accu[1]*100, '%')

C:\Users\noufa\Anaconda3\envs\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


The testing accuracy is : 80.92948794364929 %


### Pretrained Model

In [8]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

import tensorflow as tf

from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [13]:
#resize the images
IMAGE_SIZE =[224,224]

In [14]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 5s 0us/step


In [15]:
# its important that we don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [16]:
# useful for getting number of output classes
folders = glob(train_path + "/*")

In [17]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [18]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [19]:
# for viewing the structure of the model

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [21]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [23]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [24]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [25]:
len(training_set)

163

In [26]:
len(test_set)

20

In [27]:
#fit the model
#change the parameters according to your needs to get a better accuracy


r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch= len(training_set),
  validation_steps= len(test_set)
)

C:\Users\noufa\Anaconda3\envs\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
163/163 [==============================] - 1569s 10s/step - loss: 0.3605 - accuracy: 0.8625 - val_loss: 0.4795 - val_accuracy: 0.8510
Epoch 2/5
163/163 [==============================] - 1347s 8s/step - loss: 0.1609 - accuracy: 0.9403 - val_loss: 0.4123 - val_accuracy: 0.8894
Epoch 3/5
163/163 [==============================] - 1370s 8s/step - loss: 0.0880 - accuracy: 0.9646 - val_loss: 0.2389 - val_accuracy: 0.9199
Epoch 4/5
163/163 [==============================] - 2177s 13s/step - loss: 0.0921 - accuracy: 0.9657 - val_loss: 0.2774 - val_accuracy: 0.9119
Epoch 5/5
163/163 [==============================] - 1962s 12s/step - loss: 0.0768 - accuracy: 0.9726 - val_loss: 0.2619 - val_accuracy: 0.9054


In [28]:
model.save('BEST_model.h5')


In [29]:
model = load_model('BEST_model.h5')


In [32]:
#selecting an image from validation dataset for prediction

test_image = image.load_img(val_path + '/NORMAL/NORMAL2-IM-1427-0001.jpeg',target_size=[224,224])
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)

In [33]:
result

array([[0.9631918 , 0.03680819]], dtype=float32)